___

<a href='https://mainacademy.ua/'> <img src='https://mainacademy.ua/wp-content/uploads/2019/03/logo-main.png' alt = 'Хмм, щось з інтернетом'/></a>
___

# Module 4: Work with data professionally!

## Lab work 4



#### Мета: 

* навчитися обробляти дані
* "парсити" сайти

### Завдання 1:

#### Ознайомитися із API Національного банку 
https://old.bank.gov.ua/control/uk/publish/article?art_id=38441973&cat_id=38459171#exchange



1. Підключитися до API.


In [103]:
import requests

link = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange?date=20200302&json'

resp = requests.get(link).json()

2. Отримати курс долара за останній рік.

In [212]:
from datetime import datetime, timedelta, date

def get_USD_rate():
    ''' Загружает по API курс доллара за год, начиная с 1.11.20
        Возвращает список словарей USD_rates'''
    
    USD_rates = []

    start_date = date(2020, 11, 1)

    params = {'date': start_date.strftime('%Y%m%d'), 'valcode': 'USD', 'json': 'json'}
    link = 'https://bank.gov.ua/NBUStatService/v1/statdirectory/exchange'
    resp = requests.get(link, params=params).json()

    for i in range(1,366):
        resp = requests.get(link, params=params).json()
        USD_rates.extend(resp)
        params['date'] = (start_date + timedelta(days=i)).strftime('%Y%m%d')
        
    return USD_rates


3. Вивести середнє значення та відхилення курсу за кожний місяць.

In [281]:
def mean_rate_year(x):
    ''' считает среднее значение курса за год. Принимает список x, возвращае среднее значение mean_rate '''
    
    mean_rate = sum([float(r['rate']) for r in x]) / len(x)
    
    return mean_rate


def mean_rate_month(lst):
    ''' считает список средних значений курсов по месяцам, принимает список lst, возвращает список mean_month_rate '''
    
    mean_month_rate = []
    
    for m in range(1, 13):
        month_rate = []
        for rt in lst:
            if int(rt['exchangedate'][3:5]) == m:
                month_rate.append(float(rt['rate']))
            else:
                continue
            s_m = round(sum(month_rate) / len(month_rate), 2)
        mean_month_rate.append(s_m)
        
    return mean_month_rate


def deviat_month():
    '''считает отклоненение курса за каждый месяц
    lst - список средних курсов за каждый месяц, итерируемый объект
    mean_mean - среднее выборки
    возвращает словарь st_dev - отклонения за каждый месяц''' 
    
    from math import sqrt
    
    dev_dic = {}
    mean_rate = mean_rate_year(USD_rates) #среднее годовое
    lst = mean_rate_month(USD_rates)
    
    for m in range(1, 13):
        for d in lst:
            dev = sqrt(((d - mean_rate)**2) / len(lst))
            dev_dic[d] = round(dev, 2)

    
    return dev_dic


    

Средний курс: 28.22,  отклонение: 0.2
Средний курс: 27.88,  отклонение: 0.1
Средний курс: 27.8,  отклонение: 0.08
Средний курс: 27.93,  отклонение: 0.12
Средний курс: 27.6,  отклонение: 0.02
Средний курс: 27.24,  отклонение: 0.08
Средний курс: 27.21,  отклонение: 0.09
Средний курс: 26.78,  отклонение: 0.21
Средний курс: 26.73,  отклонение: 0.23
Средний курс: 26.37,  отклонение: 0.33
Средний курс: 28.31,  отклонение: 0.23
Средний курс: 28.17,  отклонение: 0.19


4. Дану інформацію записати у файл за допомогою pickle.

In [283]:
def main():
    
    dev = deviat_month()
    
    for k, v in dev.items():
        print(f'Средний курс: {k},  отклонение: {v}')

    import pickle

    with open('dev_USD_rate.pickle', 'wb') as f:
        pickle.dump(dev, f)
    
if __name__ == '__main__':
    main()
    

Средний курс: 28.22,  отклонение: 0.2
Средний курс: 27.88,  отклонение: 0.1
Средний курс: 27.8,  отклонение: 0.08
Средний курс: 27.93,  отклонение: 0.12
Средний курс: 27.6,  отклонение: 0.02
Средний курс: 27.24,  отклонение: 0.08
Средний курс: 27.21,  отклонение: 0.09
Средний курс: 26.78,  отклонение: 0.21
Средний курс: 26.73,  отклонение: 0.23
Средний курс: 26.37,  отклонение: 0.33
Средний курс: 28.31,  отклонение: 0.23
Средний курс: 28.17,  отклонение: 0.19


### Завдання 2:

Потрібно проаналізувати всі товари на сайті: 
https://smallpacking.agrosem.ua/shop/

Створіть базу даних, що містить таблицю із такими стовбцями: 
    - артикул товару
    - найменування
    - ціна 
    - вага

In [252]:
import requests
from bs4 import BeautifulSoup

headers = {
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="95", "Chromium";v="95", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://smallpacking.agrosem.ua/shop/page/2/',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
}

response = requests.get('https://smallpacking.agrosem.ua/shop/', headers=headers)


link = 'https://smallpacking.agrosem.ua/shop/'

def get_soup(link, headers):
    soup = None
    try:
        res = requests.get(link, headers=headers)
        soup = BeautifulSoup(res.text, 'lxml')
    except Exception as e:
        print(e)
    
    return soup

def qty_pages(link, headers):
    pages = []
    max_page = 0
    
    soup = get_soup(link, headers)
    
    if soup != None:       
        for page in soup.find('ul', class_='page-numbers').find_all('li'):
            page = page.find('a')
            if page != None:
                max_page = page.text
                if max_page != '':
                    max_page = int(max_page)
                    pages.append(max_page)
                else:
                    continue
            else:
                continue
        
        pages = sorted(pages, reverse = True)
        max_page = pages[0]
    
    return max_page


def get_goods(page):
    '''получает товары по всем страницам сайта'''
    
    soup = get_soup(link+f'/page/{page}', headers)
        
    if soup != None:
        for item in soup.find_all('div', class_='product-from-category-container'):
            name = item.find('a').find('span', class_='pruduct-name-bottom').text
            
            price = item.find('span', class_='regular-price').text[:-4]
            if price != None and price != '':
                price = float(price.replace(' ', ''))
            else:
                price = 0
        
            href = item.find('a').get('href')
            res_inn = requests.get(href)
            soup_inn = BeautifulSoup(res_inn.text, 'lxml')
            sku = soup_inn.find('span', itemprop='productID').text.split()[1]

            good_attr = soup_inn.find("div", class_="atributes-description")
            weight = good_attr.find("tr", {"class" : "woocommerce-product-attributes-item woocommerce-product-attributes-item--attribute_pa_vaga"}) 
    
            if weight != None : 
                weight = weight.td.p.text.replace("кг", "").replace(" ", "")
        
                if weight.isnumeric():
                    weight = float(weight)
            else:
                weight = float(0)
                            
            insert_items(sku, name, price, weight)
        
        

In [253]:
import sqlite3

sql_text = '''CREATE TABLE IF NOT EXISTS goods(
                                    sku TEXT,
                                    name TEXT NOT NULL,
                                    price FLOAT,
                                    weight FLOAT
                                    )

'''
conn = sqlite3.connect('agro_sem_db')
curs = conn.cursor()
curs.execute(sql_text)
conn.close()

In [254]:
def insert_items(sku, name, price, weight):
    sql_text = '''INSERT INTO goods (sku, name, price, weight) VALUES(?, ?, ?, ?)'''
    
    conn = sqlite3.connect('agro_sem_db')
    curs = conn.cursor()
    curs.execute(sql_text, (sku, name, price, weight))
    conn.commit()
    conn.close()


In [255]:
if __name__ == '__main__':
    
    max_page = qty_pages(link, headers)
    
    for i in range(1, max_page+1):
        get_goods(i)


### Завдання 3: 

Ознайомтеся із роботою SQLite та відповідним модулем у Python.
Завантажте базу даних для виконання лабораторної роботи.
Підключіться до завантаженої бази SQLite.

1. Виведіть інформацію про дану базу.

In [262]:
import sqlite3

newline_indent = '\n   '
conn = sqlite3.connect('tysql.sqlite')
curs = conn.cursor()

result = curs.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = sorted(list(zip(*result))[0])

for table_name in table_names:
    result = curs.execute("PRAGMA table_info('%s')" % table_name).fetchall()
    print(table_name, result, sep = '\n')


Customers
[(0, 'cust_id', 'char(10)', 1, None, 1), (1, 'cust_name', 'char(50)', 1, None, 0), (2, 'cust_address', 'char(50)', 0, None, 0), (3, 'cust_city', 'char(50)', 0, None, 0), (4, 'cust_state', 'char(5)', 0, None, 0), (5, 'cust_zip', 'char(10)', 0, None, 0), (6, 'cust_country', 'char(50)', 0, None, 0), (7, 'cust_contact', 'char(50)', 0, None, 0), (8, 'cust_email', 'char(255)', 0, None, 0)]
OrderItems
[(0, 'order_num', 'int', 1, None, 1), (1, 'order_item', 'int', 1, None, 2), (2, 'prod_id', 'char(10)', 1, None, 0), (3, 'quantity', 'int', 1, None, 0), (4, 'item_price', 'decimal(8,2)', 1, None, 0)]
Orders
[(0, 'order_num', 'int', 1, None, 1), (1, 'order_date', 'datetime', 1, None, 0), (2, 'cust_id', 'char(10)', 1, None, 0)]
Products
[(0, 'prod_id', 'char(10)', 1, None, 1), (1, 'vend_id', 'char(10)', 1, None, 0), (2, 'prod_name', 'char(255)', 1, None, 0), (3, 'prod_price', 'decimal(8,2)', 1, None, 0), (4, 'prod_desc', 'text', 0, None, 0)]
Vendors
[(0, 'vend_id', 'char(10)', 1, None, 1)

2. Виведіть перелік всіх таблиць.

In [162]:
result = curs.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
table_names = sorted(list(zip(*result))[0])
print(*table_names, sep = '\n')


Customers
OrderItems
Orders
Products
Vendors


3. Список всіх cust_id з таблиці Customers table.

In [167]:
result = curs.execute("SELECT cust_id FROM Customers;").fetchall()
cust_id = list(zip(*result))[0]
print(*cust_id, sep = '\n')


1000000001
1000000002
1000000003
1000000004
1000000005


4. Всю таблицю Customers table

In [168]:
result = curs.execute("SELECT * FROM Customers;").fetchall()
print(*result, sep = '\n')

('1000000001', 'Village Toys', '200 Maple Lane', 'Detroit', 'MI', '44444', 'USA', 'John Smith', 'sales@villagetoys.com')
('1000000002', 'Kids Place', '333 South Lake Drive', 'Columbus', 'OH', '43333', 'USA', 'Michelle Green', None)
('1000000003', 'Fun4All', '1 Sunny Place', 'Muncie', 'IN', '42222', 'USA', 'Jim Jones', 'jjones@fun4all.com')
('1000000004', 'Fun4All', '829 Riverside Drive', 'Phoenix', 'AZ', '88888', 'USA', 'Denise L. Stephens', 'dstephens@fun4all.com')
('1000000005', 'The Toy Store', '4545 53rd Street', 'Chicago', 'IL', '54545', 'USA', 'Kim Howard', None)


5. Список клієнтів (cust_names) відсортованих від Z до A

In [171]:
result = curs.execute("SELECT cust_name FROM Customers ORDER BY cust_name DESC;").fetchall()

cust_names = list(zip(*result))[0]
print(*cust_names, sep = '\n')

Village Toys
The Toy Store
Kids Place
Fun4All
Fun4All


6. Таблицю клієнтів та замовлень (cust_id і order_num) . Відсортуйте по клієнту і потім по даті замовлення

In [182]:
result = curs.execute('''
                        SELECT customers.cust_id, order_num
                        FROM customers JOIN orders ON customers.cust_id = orders.cust_id
                        ORDER BY customers.cust_id, orders.order_date;
                        
                        ''').fetchall()

print(*result, sep = '\n')

('1000000001', 20009)
('1000000001', 20005)
('1000000003', 20006)
('1000000004', 20007)
('1000000005', 20008)


7. Таблицю (на основі Items) з кількість та вартістю товару. Відсортуйте в порядку спадання по кількості та вартості

In [184]:
result = curs.execute("SELECT quantity, item_price FROM OrderItems ORDER BY quantity DESC, item_price DESC;").fetchall()
print(*result, sep = '\n')

(250, 2.49)
(250, 2.49)
(250, 2.49)
(100, 10.99)
(100, 5.49)
(100, 2.99)
(100, 2.99)
(100, 2.99)
(50, 11.49)
(50, 4.49)
(20, 5.99)
(10, 11.99)
(10, 8.99)
(10, 3.49)
(10, 3.49)
(10, 3.49)
(5, 11.99)
(5, 4.99)


8. Товар (з таблиці Products), ціна якого становить 9.49

In [186]:
result = curs.execute('''
                        SELECT prod_id, prod_name 
                        FROM Products 
                        WHERE prod_price = 9.49;
                        '''
                     ).fetchall()
print(*result, sep = '\n')

('RYL01', 'King doll')
('RYL02', 'Queen doll')


9. Виведіть назву товару та ціну, яка лежить в діапазоні від  3 до 6. Відсортуйте результат в по ціні в порядку зростання

In [202]:
result = curs.execute('''
                        SELECT prod_name, prod_price 
                        FROM Products 
                        WHERE prod_price BETWEEN 3 AND 6;
                        '''
                     ).fetchall()
print(*result, sep = '\n')

('8 inch teddy bear', 5.99)
('Fish bean bag toy', 3.49)
('Bird bean bag toy', 3.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)


10. Кількість товару, що було продано

In [189]:
result = curs.execute('SELECT sum(quantity) FROM OrderItems;').fetchall()
quant = list(zip(*result))[0]
print(*quant, sep = '\n')

1430


11. Кількість найменувань товару, ціна якого більша за 4

In [190]:
result = curs.execute("SELECT count(prod_name) FROM Products WHERE prod_price > 4;").fetchall()
quant = list(zip(*result))[0]
print(*quant, sep = '\n')

6


12. Розробіть алгоритм для виведення 3 найдорожчих товарів в базі

In [197]:
result = curs.execute("SELECT prod_name, prod_price FROM Products ORDER BY prod_price DESC LIMIT 3;").fetchall()
print(*result, sep = '\n')

('18 inch teddy bear', 11.99)
('King doll', 9.49)
('Queen doll', 9.49)


13. Підрахуйте кількість замовлень для кожного клієнта використовуючи підзапити

In [204]:
result = curs.execute('''
                        SELECT DISTINCT cst.cust_name, count(order_num)
                        FROM Orders JOIN 
                            (SELECT cust_id, cust_name FROM Customers) AS cst 
                        ON cst.cust_id = Orders.cust_id 
                        GROUP by Orders.cust_id;
                        '''
                     ).fetchall()
print(*result, sep = '\n')

('Village Toys', 2)
('Fun4All', 1)
('The Toy Store', 1)


14. Виведіть список клієнтів та їх замовлення (використайте JOIN)

In [203]:
result = curs.execute('''

                        SELECT DISTINCT customers.cust_name, orders.order_num, orderitems.prod_id, orderitems.quantity
                        FROM customers JOIN orders JOIN orderitems
                        ON customers.cust_id = orders.cust_id 
                        AND orders.order_num = orderitems.order_num;
                        
                        '''
                     ).fetchall()
print(*result, sep = '\n')

('Village Toys', 20005, 'BR01', 100)
('Village Toys', 20005, 'BR03', 100)
('Fun4All', 20006, 'BR01', 20)
('Fun4All', 20006, 'BR02', 10)
('Fun4All', 20006, 'BR03', 10)
('Fun4All', 20007, 'BR03', 50)
('Fun4All', 20007, 'BNBG01', 100)
('Fun4All', 20007, 'BNBG02', 100)
('Fun4All', 20007, 'BNBG03', 100)
('Fun4All', 20007, 'RGAN01', 50)
('The Toy Store', 20008, 'RGAN01', 5)
('The Toy Store', 20008, 'BR03', 5)
('The Toy Store', 20008, 'BNBG01', 10)
('The Toy Store', 20008, 'BNBG02', 10)
('The Toy Store', 20008, 'BNBG03', 10)
('Village Toys', 20009, 'BNBG01', 250)
('Village Toys', 20009, 'BNBG02', 250)
('Village Toys', 20009, 'BNBG03', 250)


15. Зробіть два запити: список товарів, ціна яких менша 5 та список товарів, ціна який більша рівна 5. Використайте UNION для об'єднання цих запитів

In [200]:
result = curs.execute('''
                        SELECT prod_name, prod_price
                        FROM Products WHERE prod_price < 5
                        
                        UNION
                        
                        SELECT prod_name, prod_price
                        FROM Products WHERE prod_price >= 5;
                        
                        '''
                     ).fetchall()
print(*result, sep = '\n')
conn.close()

('12 inch teddy bear', 8.99)
('18 inch teddy bear', 11.99)
('8 inch teddy bear', 5.99)
('Bird bean bag toy', 3.49)
('Fish bean bag toy', 3.49)
('King doll', 9.49)
('Queen doll', 9.49)
('Rabbit bean bag toy', 3.49)
('Raggedy Ann', 4.99)


### Завдання 4: 

Зареєструйтеся на сайті  http://www.sql-ex.ru/

Виконайте завдання із блоку SELECT; з 1 по 30 завдання

Код запитів вставляйте сюди


In [ ]:
SELECT model, speed, hd FROM PC WHERE price < 500;

In [ ]:
SELECT DISTINCT maker FROM Product WHERE type = 'Printer';

In [ ]:
SELECT model, ram, screen FROM Laptop WHERE price > 1000;

In [ ]:
SELECT * FROM Printer WHERE color = 'y';

In [ ]:
SELECT model, speed, hd FROM pc WHERE (cd = '12x' OR cd = '24x') AND price < 600;

In [ ]:
SELECT DISTINCT Product.maker, Laptop.speed
FROM Laptop JOIN Product 
ON Laptop.model = Product.model 
WHERE Laptop.hd >= 10;

In [ ]:
SELECT Product.model, PC.price FROM Product JOIN PC ON Product.model = PC.model WHERE Product.maker = 'B'
UNION
SELECT Product.model, Laptop.price FROM Product JOIN Laptop ON Product.model = Laptop.model WHERE Product.maker = 'B'
UNION
SELECT Product.model, Printer.price FROM Product JOIN Printer ON Product.model = Printer.model WHERE Product.maker = 'B';

In [ ]:
SELECT DISTINCT maker
FROM Product AS pc_product
WHERE type = 'PC' AND NOT EXISTS
    (SELECT maker 
     FROM Product 
     WHERE type = 'Laptop' AND maker = pc_product.maker);

In [ ]:
SELECT DISTINCT maker
FROM Product JOIN PC
ON Product.model = PC.model WHERE PC.speed >= 450;

In [ ]:
SELECT model, price
FROM Printer 
WHERE price IN
    (SELECT MAX(price) FROM Printer);

In [ ]:
SELECT AVG(speed) FROM PC;

In [ ]:
SELECT AVG(speed) FROM Laptop WHERE price > 1000;

In [ ]:
SELECT AVG(speed)
FROM PC
WHERE model IN 
    (SELECT model 
     FROM Product 
     WHERE maker = 'A')

In [ ]:
SELECT Ships.class, Ships.name, Classes.country 
FROM Ships LEFT JOIN Classes ON Ships.class = Classes.class 
WHERE Classes.numGuns >= 10;

In [ ]:
SELECT DISTINCT hd 
FROM PC 
GROUP BY hd 
HAVING COUNT(model) >=2;

In [ ]:
SELECT DISTINCT A.model, B.model, A.speed, A.ram 
FROM PC AS A, PC B 
WHERE A.speed = B.speed 
    AND A.ram = B.ram 
    AND A.model > B.model;

In [ ]:
SELECT DISTINCT Product.type, Laptop.model, Laptop.speed 
FROM Laptop JOIN Product 
ON Product.model = Laptop.model
WHERE Laptop.speed < ALL 
    (SELECT speed FROM PC);

In [ ]:
SELECT maker, AVG (screen) 
FROM Product JOIN Laptop 
ON Product.model = Laptop.model 
GROUP BY maker;

In [ ]:
SELECT maker, COUNT(DISTINCT model) as Count_Model 
FROM Product 
WHERE type = 'PC' 
GROUP BY maker 
HAVING COUNT(DISTINCT model) >= 3;

In [ ]:
SELECT DISTINCT maker, MAX(price) 
FROM PC JOIN Product 
ON PC.model = Product.model 
GROUP BY maker;

In [ ]:
SELECT DISTINCT speed, AVG(price) 
FROM PC 
WHERE speed > 600 
GROUP BY speed;

In [ ]:
SELECT PR.maker FROM 
    (SELECT maker 
     FROM Product JOIN PC 
     ON Product.model = PC.model 
     WHERE speed >= 750  
     
     INTERSECT 
     
     SELECT maker 
     FROM Product JOIN Laptop 
     ON Product.model = Laptop.model 
     WHERE speed >= 750 ) PR;

In [ ]:
WITH RES_OUT (model, price) AS (
    SELECT model, price FROM PC
    UNION 
    SELECT model, price FROM Laptop
    UNION 
    SELECT model, price FROM Printer )
SELECT model FROM RES_OUT 
WHERE price = ALL (SELECT MAX(price) FROM RES_OUT);

In [ ]:
SELECT AVG(Res.price) FROM (SELECT model, price FROM PC 
                                UNION
                                SELECT model, price FROM Laptop) Res 
JOIN Product ON Res.model = Product.model WHERE Product.maker = 'A';

In [ ]:
SELECT COUNT(One_Mod.maker) FROM (SELECT maker FROM Product GROUP BY maker HAVING COUNT(model) = 1) One_Mod;
